# Importando as bibliotecas:

In [ ]:
# Importando libs para processamento de texto:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import pos_tag, word_tokenize

# Importando utilitários
import re
import numpy as np
import math

from keras.preprocessing import sequence
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from gensim.models import Word2Vec

In [ ]:
# Baixndo os recursos necessários da nlkt:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Exemplos de usos da lib nlkt:

In [ ]:
# Divisão do texto em tokens:

texto = "O processamento de linguagem natural é uma área fascinante da ciência da computação."
tokens = word_tokenize(texto)
print(tokens)

['O', 'processamento', 'de', 'linguagem', 'natural', 'é', 'uma', 'área', 'fascinante', 'da', 'ciência', 'da', 'computação', '.']


In [ ]:
# Remoção das stop_words:

stop_words = set(stopwords.words('portuguese'))
filtered_sentence = [w for w in word_tokenize(texto) if not w.lower() in stop_words]
print(filtered_sentence)

['processamento', 'linguagem', 'natural', 'área', 'fascinante', 'ciência', 'computação', '.']


In [ ]:
# Stemming para reduzir as palavras a sua forma base/raiz: (facilita o processamento dos dados)

words = ["Processing", "Processes", "Processed", "Processor", "Procession", "Processional", "Processable", "Processedness", "Processibility", "Processionally"]
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in words]
print(stemmed_words)

['process', 'process', 'process', 'processor', 'process', 'procession', 'process', 'processed', 'process', 'procession']


In [ ]:
# Análise básica de sentimentos:

sia = SentimentIntensityAnalyzer()
text = "I love programming in Python!"
sentiment_score = sia.polarity_scores(text)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound': 0.6696}


In [ ]:
# Atribui a cada token uma tag de POS (Part-of-Speech), ou seja sua classe gramatical. sempre vai retornar essa tag em inglês.

text = "O processamento de linguagem natural é uma área fascinante da ciência da computação."
tagged_words = pos_tag(tokens)
print(tagged_words)

[('O', 'NNP'), ('processamento', 'NN'), ('de', 'FW'), ('linguagem', 'FW'), ('natural', 'JJ'), ('é', 'NNP'), ('uma', 'JJ'), ('área', 'NNP'), ('fascinante', 'NN'), ('da', 'NN'), ('ciência', 'NN'), ('da', 'NN'), ('computação', 'NN'), ('.', '.')]


# Processando um texto completo:

In [ ]:
texto = """No meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra.
Nunca me esquecerei desse acontecimento
na vida de minhas retinas tão fatigadas.
Nunca me esquecerei que no meio do caminho
tinha uma pedra
tinha uma pedra no meio do caminho
no meio do caminho tinha uma pedra."""

print(texto)

No meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra.
Nunca me esquecerei desse acontecimento
na vida de minhas retinas tão fatigadas.
Nunca me esquecerei que no meio do caminho
tinha uma pedra
tinha uma pedra no meio do caminho
no meio do caminho tinha uma pedra.


In [ ]:
texto = texto.lower()
print(texto)

"""
Colocar todas as letras em minúsculo é importante
porque nossa máquina tende a interpretar palavras iguais
mas com letras minúsculas e maiúsculas como sendo diferentes.
"""

no meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra.
nunca me esquecerei desse acontecimento
na vida de minhas retinas tão fatigadas.
nunca me esquecerei que no meio do caminho
tinha uma pedra
tinha uma pedra no meio do caminho
no meio do caminho tinha uma pedra.


'\nColocar todas as letras em minúsculo é importante\nporque nossa máquina tende a interpretar palavras iguais\nmas com letras minúsculas e maiúsculas como sendo diferentes.\n'

In [ ]:
# Seleciona apenas letras (lembrando que o texto está em português e as letras podem possuir acento)
apenas_letras = re.findall(r'[a-zéóáêâãõç]+', texto)

print(apenas_letras)

['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'nunca', 'me', 'esquecerei', 'desse', 'acontecimento', 'na', 'vida', 'de', 'minhas', 'retinas', 'tão', 'fatigadas', 'nunca', 'me', 'esquecerei', 'que', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']


In [ ]:
# Junta o texto, já que o (.findall) separa em tokens
novo_texto = " ".join(apenas_letras)

print(novo_texto)

no meio do caminho tinha uma pedra tinha uma pedra no meio do caminho tinha uma pedra no meio do caminho tinha uma pedra nunca me esquecerei desse acontecimento na vida de minhas retinas tão fatigadas nunca me esquecerei que no meio do caminho tinha uma pedra tinha uma pedra no meio do caminho no meio do caminho tinha uma pedra


# Algoritmo BoW (Bag of Words):

### Gerando o vocabulário:



1.   Separar nosso texto em tokens;

2.   Criar uma lista para guardarmos o vocabulário;

3.   Fazer um loop para percorrer o texto inteiro;

4.   Criar uma condicional para verificar se a palavra está na lista —fazemos isso porque nosso vocabulário conta apenas as ocorrências únicas, sem repetições de palavras;

5.   Caso não esteja, ela é adicionada.

In [ ]:
tokens_texto = word_tokenize(novo_texto)

In [ ]:
def vocab_maker(tokens):
  Vocab = []
  for token in tokens:
      if token not in Vocab:
          Vocab.append(token)
  return Vocab

In [ ]:
vocab = vocab_maker(tokens_texto)

print(vocab)

['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'nunca', 'me', 'esquecerei', 'desse', 'acontecimento', 'na', 'vida', 'de', 'minhas', 'retinas', 'tão', 'fatigadas', 'que']


### Codificação binária:

1. Criar uma lista que representa o vetor;
2. Fazer um loop para percorrer todas as palavras do vocabulário;
3. Se a palavra estiver no documento, adicionar 1 à lista; caso contrário,
   adicionar 0;
4. Transformar a lista final em um array do numpy e retornar.

In [ ]:
def cria_vetor_documento(documento, vocab):
  vetor = []

  for palavra in vocab:
    if palavra in documento:
        vetor.append(1)
    else:
        vetor.append(0)

  return np.array(vetor)

### Exemplo de uso em uma amostra:

In [ ]:
texto_amostra = "no meio do caminho tinha uma pedra"
vetor_doc = cria_vetor_documento(texto_amostra, vocab)

print(vetor_doc)

[1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0]


# Term Frequency Inverse Document Frequency (TF-IDF):

Term Frequency mede a frequência com que um termo ocorre num documento;

Inverse Document Frequency mede o quão importante um termo é no contexto de todos os documentos.

In [ ]:
estrofe1= """no meio do caminho tinha uma pedra tinha uma pedra no meio do caminho tinha uma pedra no meio do caminho tinha uma pedra"""

estrofe2 = """nunca me esquecerei desse acontecimento na vida de minhas retinas tão fatigadas nunca me esquecerei que no meio do caminho tinha uma pedra tinha uma pedra no meio do caminho no meio do caminho tinha uma pedra"""

In [ ]:
e1_tokens = word_tokenize(estrofe1)
print(e1_tokens)

e2_tokens = word_tokenize(estrofe2)
print(e2_tokens)

['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']
['nunca', 'me', 'esquecerei', 'desse', 'acontecimento', 'na', 'vida', 'de', 'minhas', 'retinas', 'tão', 'fatigadas', 'nunca', 'me', 'esquecerei', 'que', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']


In [ ]:
tokens_uniao = e1_tokens + e2_tokens
print(tokens_uniao)

['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'nunca', 'me', 'esquecerei', 'desse', 'acontecimento', 'na', 'vida', 'de', 'minhas', 'retinas', 'tão', 'fatigadas', 'nunca', 'me', 'esquecerei', 'que', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra']


In [ ]:
novo_vocab = vocab_maker(tokens_uniao)
print(novo_vocab)

['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', 'nunca', 'me', 'esquecerei', 'desse', 'acontecimento', 'na', 'vida', 'de', 'minhas', 'retinas', 'tão', 'fatigadas', 'que']


In [ ]:
'''Recebe uma lista com o vocabulario e uma lista de tokens de um documento.
Retorna um dicionario com o numero de vezes que cada palavra do vocabulario
ocorre no documento.'''

def dicionario_de_contagem(vocabulario, documento):
  dic = dict.fromkeys(vocabulario, 0)
  for palavra in documento:
    dic[palavra] += 1

  return dic

In [ ]:
e1_dic_cont = dicionario_de_contagem(novo_vocab, e1_tokens)
e2_dic_cont = dicionario_de_contagem(novo_vocab, e2_tokens)

print(e1_dic_cont,'\n')
print(e2_dic_cont)

{'no': 3, 'meio': 3, 'do': 3, 'caminho': 3, 'tinha': 4, 'uma': 4, 'pedra': 4, 'nunca': 0, 'me': 0, 'esquecerei': 0, 'desse': 0, 'acontecimento': 0, 'na': 0, 'vida': 0, 'de': 0, 'minhas': 0, 'retinas': 0, 'tão': 0, 'fatigadas': 0, 'que': 0} 

{'no': 3, 'meio': 3, 'do': 3, 'caminho': 3, 'tinha': 3, 'uma': 3, 'pedra': 3, 'nunca': 2, 'me': 2, 'esquecerei': 2, 'desse': 1, 'acontecimento': 1, 'na': 1, 'vida': 1, 'de': 1, 'minhas': 1, 'retinas': 1, 'tão': 1, 'fatigadas': 1, 'que': 1}


### Calculando o Term Frequency:

In [ ]:
def calculaTF(dic_de_cont, doc):
  tf_dic = {}

  num_palavras_doc = len(doc)
  for palavra, contagem in dic_de_cont.items():
      tf_dic[palavra] = contagem/float(num_palavras_doc)

  return(tf_dic)

In [ ]:
e1_tf_bow = calculaTF(e1_dic_cont, e1_tokens)
e2_tf_bow = calculaTF(e2_dic_cont, e2_tokens)

In [ ]:
e1_tf_bow

{'no': 0.125,
 'meio': 0.125,
 'do': 0.125,
 'caminho': 0.125,
 'tinha': 0.16666666666666666,
 'uma': 0.16666666666666666,
 'pedra': 0.16666666666666666,
 'nunca': 0.0,
 'me': 0.0,
 'esquecerei': 0.0,
 'desse': 0.0,
 'acontecimento': 0.0,
 'na': 0.0,
 'vida': 0.0,
 'de': 0.0,
 'minhas': 0.0,
 'retinas': 0.0,
 'tão': 0.0,
 'fatigadas': 0.0,
 'que': 0.0}

In [ ]:
e2_tf_bow

{'no': 0.08108108108108109,
 'meio': 0.08108108108108109,
 'do': 0.08108108108108109,
 'caminho': 0.08108108108108109,
 'tinha': 0.08108108108108109,
 'uma': 0.08108108108108109,
 'pedra': 0.08108108108108109,
 'nunca': 0.05405405405405406,
 'me': 0.05405405405405406,
 'esquecerei': 0.05405405405405406,
 'desse': 0.02702702702702703,
 'acontecimento': 0.02702702702702703,
 'na': 0.02702702702702703,
 'vida': 0.02702702702702703,
 'de': 0.02702702702702703,
 'minhas': 0.02702702702702703,
 'retinas': 0.02702702702702703,
 'tão': 0.02702702702702703,
 'fatigadas': 0.02702702702702703,
 'que': 0.02702702702702703}

### Calculando o Inverse Document Frequency:

In [ ]:
def computaIDF(lista_de_docs):
    idf_dic = {}
    N = len(lista_de_docs)

    for palavra in lista_de_docs[0]:
        num_docs_aparece = 0
        for doc in lista_de_docs:
            if doc[palavra]>0:
                num_docs_aparece += 1

        idf_dic[palavra] = math.log10(N / (num_docs_aparece))

    return (idf_dic)

In [ ]:
estrofes_idf = computaIDF([e1_dic_cont, e2_dic_cont])
print(estrofes_idf)

{'no': 0.0, 'meio': 0.0, 'do': 0.0, 'caminho': 0.0, 'tinha': 0.0, 'uma': 0.0, 'pedra': 0.0, 'nunca': 0.3010299956639812, 'me': 0.3010299956639812, 'esquecerei': 0.3010299956639812, 'desse': 0.3010299956639812, 'acontecimento': 0.3010299956639812, 'na': 0.3010299956639812, 'vida': 0.3010299956639812, 'de': 0.3010299956639812, 'minhas': 0.3010299956639812, 'retinas': 0.3010299956639812, 'tão': 0.3010299956639812, 'fatigadas': 0.3010299956639812, 'que': 0.3010299956639812}


# Token Sequence:

In [ ]:
# Vocabulário para nossa tokenização:
palavras = ['gato', 'comeu', 'peixe', 'mesa', 'cachorro', 'da', 'mesa', 'osso', 'mordeu', 'o', 'um', 'na', 'enquanto']

# Tamanho do vocabulário (pode definir a dimensão de imput do embeddings dentro de um modelo)
tam_vocab = len(palavras)

# Frases para testar nossa tokenização
frases = []
frases.append('o gato comeu um peixe na mesa')
frases.append('o cachorro mordeu o osso')

# Instânciando o tokenizador e dando o vocabulário de entrada
tokenizer = Tokenizer(num_words=tam_vocab)
tokenizer.fit_on_texts(palavras)

# Tokenizando os textos:
frases_tokenizadas = tokenizer.texts_to_sequences(frases)

In [ ]:
# Visualizando a estrutura do tokenizer
print("Índices das palavras:")
print(tokenizer.word_index)

Índices das palavras:
{'mesa': 1, 'gato': 2, 'comeu': 3, 'peixe': 4, 'cachorro': 5, 'da': 6, 'osso': 7, 'mordeu': 8, 'o': 9, 'um': 10, 'na': 11, 'enquanto': 12}


### Verificando e corrindo a dimensionalidade dos vetores(frases):

In [ ]:
frases_tokenizadas[0]

[9, 2, 3, 10, 4, 11, 1]

In [ ]:
# Visualizando os vetores das frases:
frases_tokenizadas[1]

[9, 5, 8, 9, 7]

In [ ]:
# Achando a frase com maior número de tokens:
maior = max(len(tokens) for tokens in frases_tokenizadas)
print("Maior frase:", maior)

Maior frase: 7


In [ ]:
# Performando padding para deixar todas as "frases" com o mesmo tamanho: (adiciona 0 para completar o vetor)
frases_tokenizadas_padding = pad_sequences(frases_tokenizadas, maxlen=maior, padding='post',)

In [ ]:
frases_tokenizadas_padding[0]

array([ 9,  2,  3, 10,  4, 11,  1], dtype=int32)

In [ ]:
frases_tokenizadas_padding[1]

array([9, 5, 8, 9, 7, 0, 0], dtype=int32)

# Word-Embeddings: (word2vec)

Word2Vec é um modelo de aprendizado de máquina usado para representar palavras como vetores numéricos densos. Ele captura relações semânticas e sintáticas entre palavras. O modelo utiliza uma rede neural para aprender representações distribuídas de palavras a partir de contextos onde elas aparecem.

In [ ]:
# Texto de exemplo
texto_exemplo = "I love coding. Word embeddings are awesome. Machine learning is fun."

# Tokenização
tokens_exemplo = re.findall(r'[a-zA-Z]+', texto_exemplo)
print("Tokens:", tokens_exemplo)

Tokens: ['I', 'love', 'coding', 'Word', 'embeddings', 'are', 'awesome', 'Machine', 'learning', 'is', 'fun']


In [ ]:
# Tipo do word2vec para se utilizar: (0) cbow - (1) skip-gram
tipo = 1

# Treinamento do modelo Word2Vec
model = Word2Vec([tokens_exemplo], vector_size=100, window=5, min_count=1, workers=4, sg=tipo)

In [ ]:
# Obter o vetor de uma palavra
vector = model.wv['coding']

print("Vetor de 'coding':", vector)

Vetor de 'coding': [-9.5785465e-03  8.9431154e-03  4.1650687e-03  9.2347348e-03
  6.6435025e-03  2.9247368e-03  9.8040197e-03 -4.4246409e-03
 -6.8033109e-03  4.2273807e-03  3.7290000e-03 -5.6646108e-03
  9.7047603e-03 -3.5583067e-03  9.5494064e-03  8.3472609e-04
 -6.3384566e-03 -1.9771170e-03 -7.3770545e-03 -2.9795230e-03
  1.0416972e-03  9.4826873e-03  9.3558477e-03 -6.5958775e-03
  3.4751510e-03  2.2755705e-03 -2.4893521e-03 -9.2291720e-03
  1.0271263e-03 -8.1657059e-03  6.3201892e-03 -5.8000805e-03
  5.5354391e-03  9.8337233e-03 -1.6000033e-04  4.5284927e-03
 -1.8094003e-03  7.3607611e-03  3.9400971e-03 -9.0103243e-03
 -2.3985039e-03  3.6287690e-03 -9.9568366e-05 -1.2012708e-03
 -1.0554385e-03 -1.6716016e-03  6.0495257e-04  4.1650953e-03
 -4.2527914e-03 -3.8336217e-03 -5.2816868e-05  2.6935578e-04
 -1.6880632e-04 -4.7855065e-03  4.3134023e-03 -2.1719194e-03
  2.1035396e-03  6.6652300e-04  5.9696771e-03 -6.8423809e-03
 -6.8157101e-03 -4.4762576e-03  9.4358288e-03 -1.5918827e-03
 -9.4

In [ ]:
# Encontrar palavras similares
similar_words = model.wv.most_similar('coding')

print("Palavras similares a 'coding':", similar_words)

Palavras similares a 'coding': [('learning', 0.19912061095237732), ('Machine', 0.07497556507587433), ('are', 0.060591842979192734), ('love', 0.04469989985227585), ('is', 0.03364057466387749), ('fun', 0.027057481929659843), ('I', 0.026806799694895744), ('embeddings', 0.008826158009469509), ('awesome', -0.06900332123041153), ('Word', -0.14454564452171326)]


### Realizando um novo treinamento sob o nosso modelo:

In [ ]:
# Treinamento com mais dados (opcional)
mais_texto = "Deep learning is a powerful technique. Natural language processing is fascinating."
mais_tokens = re.findall(r'[a-zA-Z]+', mais_texto)

model.build_vocab([mais_tokens], update=True)
model.train([mais_tokens], total_examples=model.corpus_count, epochs=model.epochs)

(4, 55)

In [ ]:
# Encontrar palavras similares
similar_words = model.wv.most_similar('coding')
print("Palavras similares a 'coding':", similar_words)

Palavras similares a 'coding': [('learning', 0.19912061095237732), ('Machine', 0.07497556507587433), ('are', 0.060591842979192734), ('love', 0.04469989985227585), ('Deep', 0.0377129502594471), ('is', 0.03364057466387749), ('fun', 0.027057481929659843), ('I', 0.026806797832250595), ('fascinating', 0.015797005966305733), ('processing', 0.01243623998016119)]


### Salvando nosso modelo:

In [ ]:
# Salvar modelo treinado
model.save("word2vec_model.bin")

# Carregar modelo treinado
loaded_model = Word2Vec.load("word2vec_model.bin")

### Calculando algumas propriedades:

In [ ]:
# Calculando a similaridade entre duas palavras
similarity_score = loaded_model.wv.similarity('Deep', 'learning')

print("Similaridade entre as palavras escolhidas é: ", similarity_score)

Similaridade entre as palavras escolhidas é:  0.17272793


In [ ]:
# Calculando a distância entre dois vetores de palavra
distance = loaded_model.wv.distance('Deep', 'learning')
print("Distância entre as palavras escolhidas é: ", distance)

Distância entre as palavras escolhidas é:  0.8272720724344254
